<a href="https://colab.research.google.com/github/marioschuele/masterarbeit/blob/main/SOTA_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img
import os
import numpy as np
import pandas as pd

In [3]:
# Install packages
! pip install kaggle -q

In [4]:
#Download and unzip Kaggle dataset
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download yuweisunut/sidd-segmented-intrusion-detection-dataset
!unzip sidd-segmented-intrusion-detection-dataset

mkdir: cannot create directory ‘/root/.kaggle’: File exists
sidd-segmented-intrusion-detection-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  sidd-segmented-intrusion-detection-dataset.zip
replace SIDD/n005/local.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: N

N


In [5]:
directory = 'SIDD'

imgs = {}
uid = 0

for client in os.listdir(directory):
  curr_path = f'{directory}/{client}/pcap'

  for subdir in os.listdir(curr_path):
    curr_path = f'{directory}/{client}/pcap/{subdir}/dataset'
    curr_type = subdir[-1:]
    
    for dayscen in os.listdir(curr_path):
      curr_path = f'{directory}/{client}/pcap/{subdir}/dataset/{dayscen}'
      #+print(dayscen)
      for img in os.listdir(curr_path):
        if dayscen == 'benign':
            imgs[uid] = {'id': uid, 'label': str(0), 'fn': img, 'path': curr_path + '/' + img}
        elif dayscen == 'malicious':
            imgs[uid] = {'id': uid, 'label': str(curr_type), 'fn': img, 'path': curr_path + '/' + img}
        uid +=1


In [6]:
img_df = pd.DataFrame.from_dict(imgs,orient='index')
img_df['label'] = img_df['label'].astype(int)
img_df.loc[img_df.index[(img_df['label']==3)],'label'] = 2

In [7]:
def _parse_function(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image_decoded, tf.float32)
    return image, label

file_paths = img_df.path
file_labels = img_df["label"]

dataset = tf.data.Dataset.from_tensor_slices((file_paths, file_labels))

dataset = dataset.map(_parse_function)
dataset = dataset.batch(32)

#iterator = tf.compat.v1.data.make_one_shot_iterator(dataset)
#images, labels = iterator.get_next()
#labels


In [8]:
#Distribute dataset into train, validation and test set
def get_dataset_partitions_tf(ds, ds_size, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    if shuffle:
        # Specify seed to always have the same split distribution between runs
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    print('Size of {}:  {}'.format('train ds', str(train_ds.cardinality().numpy())))
    print('Size of {}:  {}'.format('val ds', str(val_ds.cardinality().numpy())))
    print('Size of {}:  {}'.format('test ds', str(test_ds.cardinality().numpy())))
    
    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = get_dataset_partitions_tf(ds=dataset, ds_size=dataset.cardinality().numpy(), train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000)

Size of train ds:  34640
Size of val ds:  4330
Size of test ds:  4330


In [ ]:
model = tf.keras.applications.EfficientNetV2M(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=3,
)

model.compile(
    optimizer=keras.optimizers.SGD(),  
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']        #keras.metrics.sparse_categorical_accuracy
)


history = model.fit(
    x=train_ds,
    epochs=10,
    validation_data=val_ds
)

evaluation = model.evaluate(test_ds)

In [2]:
#Plot accuracy of model
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#Plot loss of model
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: ignored

In [ ]:
nodes = ['n005', 'n006', 'n008', 'n031', 'n034', 'n035', 'n036', 'n038', 'n041', 'n045', 'n046', 'n047', 'n048', 'n053', 'n056']
classes = [0,1,3]#['Benign', 'Server Message Block', 'TCP SYN flood attack']

image_size = (384,384)
batch_size = None

train_datagen = ImageDataGenerator(validation_split=0.2)
train_batches = train_datagen.flow_from_dataframe(
    img_df,
    x_col="path",
    y_col="type",
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = 'training'
)

validation_batches = train_datagen.flow_from_dataframe(
    img_df,
    x_col="path",
    y_col="type",
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    subset = 'validation'
)

test_batches = train_datagen.flow_from_dataframe(
    img_df,
    x_col="path",
    y_col="type",
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = False,
    subset = 'validation'
)



Found 1108468 validated image filenames belonging to 3 classes.
Found 277116 validated image filenames belonging to 3 classes.
Found 277116 validated image filenames belonging to 3 classes.
